In [1]:
import numpy as np
import pandas as pd
import math_functions as mf

In [2]:
# set random seed
np.random.seed(1234)

# parameters
n_time_points = 1000 # number of time points in the growth curve
t_max = 200
time_points_range = np.array([np.around(x, 3) for x in np.linspace(0, t_max, n_time_points).tolist()])


---
## logistic model, no noise
### blanks

In [3]:
measurement_noise = 0.0

In [4]:
# generate blanks
n_blanks = 3
blank_avg_val = 0.1


df_blanks = pd.DataFrame(columns = ['N0', 'A', 'mu_max', 'l', 'sample_name'] + time_points_range.tolist())


for i in range(n_blanks):
    y = [blank_avg_val + np.around(np.random.normal(0, measurement_noise)) for x in range(n_time_points)]
    df_blanks.loc[i] = [np.nan, np.nan, np.nan, np.nan, 'blank {}'.format(i)] + y

df_blanks_mean = df_blanks.iloc[:, 5:].mean(axis = 0)



In [5]:
# generate random Logistic growth curves
n = 100 # number of curves to generate



out = pd.DataFrame(columns=['N0', 'A', 'mu_max', 'l', 'sample_name'] + time_points_range.tolist())


for i in range(n):
    # generate random parameters
    N0 = np.around(np.random.uniform(0, 0.5), 4)
    A = np.around(np.random.uniform(0.5, 4), 4)
    mu_max = np.around(np.random.uniform(0, 2), 4)
    l = np.around(np.random.uniform(0, 100), 4)



    # generate growth curve
    y_log = mf.modified_logistic(time_points_range, N0, A, mu_max, l)
    y = np.exp(y_log)

    # add noise to the growth curve
    y = [y_i + np.random.normal(0, measurement_noise) for y_i in y]

    # add blank to the growth curve
    y = np.array(y) + df_blanks_mean.values


    out.loc[i] = [N0, A, mu_max, l, 'sample {}'.format(i)] + y.tolist()



out_complete = pd.concat([df_blanks, out], axis=0).reset_index(drop=True)
out_complete.to_excel('./synthetic_logistic_growth_curves.xlsx', index=False)
out_complete_clean = out_complete.iloc[:, 4:]
out_complete_clean.to_excel('./synthetic_logistic_growth_curves_dgc_ready.xlsx', index=False)  # drop the first 4 columns



/home/m/Desktop/dashing_growth_curves/growthdash/implementation_test/math_functions.py:29: RuntimeWarning: overflow encountered in power
  return np.log(N0) + A * 1 / (1 + np.exp(1)**(4 * mu / A * (l - x) + 2))


### compare to Dashing Growth Curve fits
Fit synthetic growth data with Dashing Growth Curves (conventional Logistic function) and determine the error between estimated and true parameters.

As a sanity check, fit synthetic data also with the conventional Gompertz model. The Gompertz fits should be worse than the Logistic fits.

In [20]:
# load data exported from Dashing Growth Curves:
df_logistic_fit = pd.read_csv('./synthetic_logistic_data_logistic_fits.csv', sep='\t').iloc[n_blanks:, :]
df_gompertz_fit = pd.read_csv('./synthetic_logistic_data_gompertz_fits.csv', sep='\t').iloc[n_blanks:, :]
out_complete_noblanks = pd.read_excel('./synthetic_logistic_growth_curves.xlsx').iloc[n_blanks:, :]


# error functions
def compute_rmse(df_1, df_2, column_name_1, column_name_2):
    return np.sqrt((df_1[column_name_1] - df_2[column_name_2]).pow(2).sum()/df_1.shape[0])

def compute_median_relative_error(df_1, df_2, column_name_1, column_name_2):
    # return np.abs((df_1[column_name_1] - df_2[column_name_2]) / df_2[column_name_2]).sum()/df_1.shape[0]
    return np.median(np.abs((df_1[column_name_1] - df_2[column_name_2]) / df_2[column_name_2]))



df_analysis = pd.DataFrame(columns = ['parameter', 'average_relative_error_logistic_model', 'average_relative_error_gompertz_model'])

# compute errors in N0
N0_log = compute_median_relative_error(df_logistic_fit, out_complete_noblanks, 'N0', 'N0')
N0_gomp = compute_median_relative_error(df_gompertz_fit, out_complete_noblanks, 'N0', 'N0')
print('the relative fitting error of N0 is {:.4f} for logistic fits and {:.4f} for Gompertz fits'.format(N0_log, N0_gomp))
df_analysis.loc[0] = ['N0', N0_log, N0_gomp]

# compute errors in A
A_log = compute_median_relative_error(df_logistic_fit, out_complete_noblanks, 'A', 'A')
A_gomp = compute_median_relative_error(df_gompertz_fit, out_complete_noblanks, 'A', 'A')
print('the relative fitting error of A is {:.4f} for logistic fits and {:.4f} for Gompertz fits'.format(A_log, A_gomp))
df_analysis.loc[1] = ['A', A_log, A_gomp]

# compute errors in mu_max
mu_log = compute_median_relative_error(df_logistic_fit, out_complete_noblanks, 'mumax', 'mu_max')
mu_gomp = compute_median_relative_error(df_gompertz_fit, out_complete_noblanks, 'mumax', 'mu_max')
print('the relative fitting error of mu_max is {:.4f} for logistic fits and {:.4f} for Gompertz fits'.format(A_log, A_gomp))
df_analysis.loc[2] = ['mu_max', mu_log, mu_gomp]

# compute errors in l
l_log = compute_median_relative_error(df_logistic_fit, out_complete_noblanks, 't0', 'l')
l_gomp = compute_median_relative_error(df_gompertz_fit, out_complete_noblanks, 't0', 'l')
print('the relative fitting error of l is {:.4f} for logistic fits and {:.4f} for Gompertz fits'.format(l_log, l_gomp))
df_analysis.loc[3] = ['l', l_log, l_gomp]


df_analysis = df_analysis.round(4)
df_analysis.to_excel('./synthetic_logistic_growth_curves_analysis.xlsx', index=False)
df_analysis

the relative fitting error of N0 is 0.0000 for logistic fits and 0.0070 for Gompertz fits
the relative fitting error of A is 0.0000 for logistic fits and 0.0027 for Gompertz fits
the relative fitting error of mu_max is 0.0000 for logistic fits and 0.0027 for Gompertz fits
the relative fitting error of l is 0.0000 for logistic fits and 0.0071 for Gompertz fits


,parameter,average_relative_error_logistic_model,average_relative_error_gompertz_model
0,N0,0.0,0.0070
1,A,0.0,0.0027
2,mu_max,0.0,0.0348
3,l,0.0,0.0071


# logistic model, with noise

In [22]:
out_complete_clean = pd.read_excel('./synthetic_logistic_growth_curves.xlsx').iloc[:, 4:]


In [30]:
measurement_noise = 0.05
out_complete_clean_noised = out_complete_clean.copy()
shape_noise = [out_complete_clean_noised.shape[0], out_complete_clean_noised.shape[1] - 1]
out_complete_clean_noised.iloc[:, 1:] = out_complete_clean_noised.iloc[:, 1:] + np.random.normal(0, measurement_noise, shape_noise)
out_complete_clean_noised.to_excel('./synthetic_logistic_growth_curves_dgc_ready_noised.xlsx', index=False)

### compare to Dashing Growth Curve fits
Fit synthetic growth data with Dashing Growth Curves (conventional Logistic function) and determine the error between estimated and true parameters.

As a sanity check, fit synthetic data also with the conventional Gompertz model. The Gompertz fits should be worse than the Logistic fits.

In [32]:
# load data exported from Dashing Growth Curves:
df_logistic_fit = pd.read_csv('./synthetic_logistic_data_noised_logistic_fits.csv', sep='\t').iloc[n_blanks:, :]
df_gompertz_fit = pd.read_csv('./synthetic_logistic_data_noised_gompertz_fits.csv', sep='\t').iloc[n_blanks:, :]
out_complete_noblanks = pd.read_excel('./synthetic_logistic_growth_curves.xlsx').iloc[n_blanks:, :]


# error functions
def compute_rmse(df_1, df_2, column_name_1, column_name_2):
    return np.sqrt((df_1[column_name_1] - df_2[column_name_2]).pow(2).sum()/df_1.shape[0])

def compute_median_relative_error(df_1, df_2, column_name_1, column_name_2):
    # return np.abs((df_1[column_name_1] - df_2[column_name_2]) / df_2[column_name_2]).sum()/df_1.shape[0]
    return np.median(np.abs((df_1[column_name_1] - df_2[column_name_2]) / df_2[column_name_2]))


df_analysis = pd.DataFrame(columns = ['parameter', 'average_relative_error_logistic_model', 'average_relative_error_gompertz_model'])

# compute errors in N0
N0_log = compute_median_relative_error(df_logistic_fit, out_complete_noblanks, 'N0', 'N0')
N0_gomp = compute_median_relative_error(df_gompertz_fit, out_complete_noblanks, 'N0', 'N0')
print('the relative fitting error of N0 is {:.4f} for logistic fits and {:.4f} for Gompertz fits'.format(N0_log, N0_gomp))
df_analysis.loc[0] = ['N0', N0_log, N0_gomp]

# compute errors in A
A_log = compute_median_relative_error(df_logistic_fit, out_complete_noblanks, 'A', 'A')
A_gomp = compute_median_relative_error(df_gompertz_fit, out_complete_noblanks, 'A', 'A')
print('the relative fitting error of A is {:.4f} for logistic fits and {:.4f} for Gompertz fits'.format(A_log, A_gomp))
df_analysis.loc[1] = ['A', A_log, A_gomp]

# compute errors in mu_max
mu_log = compute_median_relative_error(df_logistic_fit, out_complete_noblanks, 'mumax', 'mu_max')
mu_gomp = compute_median_relative_error(df_gompertz_fit, out_complete_noblanks, 'mumax', 'mu_max')
print('the relative fitting error of mu_max is {:.4f} for logistic fits and {:.4f} for Gompertz fits'.format(A_log, A_gomp))
df_analysis.loc[2] = ['mu_max', mu_log, mu_gomp]

# compute errors in l
l_log = compute_median_relative_error(df_logistic_fit, out_complete_noblanks, 't0', 'l')
l_gomp = compute_median_relative_error(df_gompertz_fit, out_complete_noblanks, 't0', 'l')
print('the relative fitting error of l is {:.4f} for logistic fits and {:.4f} for Gompertz fits'.format(l_log, l_gomp))
df_analysis.loc[3] = ['l', l_log, l_gomp]


df_analysis = df_analysis.round(4)
df_analysis.to_excel('./synthetic_logistic_growth_curves_noised_analysis.xlsx', index=False)
df_analysis

the relative fitting error of N0 is 0.0366 for logistic fits and 0.0338 for Gompertz fits
the relative fitting error of A is 0.0227 for logistic fits and 0.0219 for Gompertz fits
the relative fitting error of mu_max is 0.0227 for logistic fits and 0.0219 for Gompertz fits
the relative fitting error of l is 0.0025 for logistic fits and 0.0036 for Gompertz fits


,parameter,average_relative_error_logistic_model,average_relative_error_gompertz_model
0,N0,0.0366,0.0338
1,A,0.0227,0.0219
2,mu_max,0.0655,0.0754
3,l,0.0025,0.0036


---
# gompertz model, no noise

In [34]:
measurement_noise = 0

In [33]:
# generate random Logistic growth curves
n = 100 # number of curves to generate

out = pd.DataFrame(columns=['N0', 'A', 'mu_max', 'l', 'sample_name'] + time_points_range.tolist())


for i in range(n):
    # generate random parameters
    N0 = np.around(np.random.uniform(0, 0.5), 4)
    A = np.around(np.random.uniform(0.5, 4), 4)
    mu_max = np.around(np.random.uniform(0, 2), 4)
    l = np.around(np.random.uniform(0, 100), 4)


    # generate growth curve
    y_log = mf.modified_gompertz(time_points_range, N0, A, mu_max, l)
    y = np.exp(y_log)

    # add noise to the growth curve
    y = [y_i + np.random.normal(0, measurement_noise) for y_i in y]

    # add blank to the growth curve
    y = np.array(y) + df_blanks_mean.values


    out.loc[i] = [N0, A, mu_max, l, 'sample {}'.format(i)] + y.tolist()



out_complete = pd.concat([df_blanks, out], axis=0).reset_index(drop=True)
out_complete.to_excel('./synthetic_gompertz_growth_curves.xlsx', index=False)
out_complete_clean = out_complete.iloc[:, 4:]
out_complete_clean.to_excel('./synthetic_gompertz_growth_curves_dgc_ready.xlsx', index=False)  # drop the first 4 columns



### compare to Dashing Growth Curve fits
Fit synthetic growth data with Dashing Growth Curves (conventional Logistic function) and determine the error between estimated and true parameters.

As a sanity check, fit synthetic data also with the conventional Logistic model. The Logistic fits should be worse than the Gompertz fits.

In [10]:
# load data exported from Dashing Growth Curves:
df_logistic_fit = pd.read_csv('./synthetic_gompertz_data_logistic_fits.csv', sep='\t').iloc[n_blanks:, :]
df_gompertz_fit = pd.read_csv('./synthetic_gompertz_data_gompertz_fits.csv', sep='\t').iloc[n_blanks:, :]
out_complete_noblanks = pd.read_excel('./synthetic_gompertz_growth_curves.xlsx').iloc[n_blanks:, :]


# error functions
def compute_rmse(df_1, df_2, column_name_1, column_name_2):
    return np.sqrt((df_1[column_name_1] - df_2[column_name_2]).pow(2).sum()/df_1.shape[0])

def compute_median_relative_error(df_1, df_2, column_name_1, column_name_2):
    # return np.abs((df_1[column_name_1] - df_2[column_name_2]) / df_2[column_name_2]).sum()/df_1.shape[0]
    return np.median(np.abs((df_1[column_name_1] - df_2[column_name_2]) / df_2[column_name_2]))



df_analysis = pd.DataFrame(columns = ['parameter', 'average_relative_error_logistic_model', 'average_relative_error_gompertz_model'])

# compute errors in N0
N0_log = compute_median_relative_error(df_logistic_fit, out_complete_noblanks, 'N0', 'N0')
N0_gomp = compute_median_relative_error(df_gompertz_fit, out_complete_noblanks, 'N0', 'N0')
print('the relative fitting error of N0 is {:.4f} for logistic fits and {:.4f} for Gompertz fits'.format(N0_log, N0_gomp))
df_analysis.loc[0] = ['N0', N0_log, N0_gomp]

# compute errors in A
A_log = compute_median_relative_error(df_logistic_fit, out_complete_noblanks, 'A', 'A')
A_gomp = compute_median_relative_error(df_gompertz_fit, out_complete_noblanks, 'A', 'A')
print('the relative fitting error of A is {:.4f} for logistic fits and {:.4f} for Gompertz fits'.format(A_log, A_gomp))
df_analysis.loc[1] = ['A', A_log, A_gomp]

# compute errors in mu_max
mu_log = compute_median_relative_error(df_logistic_fit, out_complete_noblanks, 'mumax', 'mu_max')
mu_gomp = compute_median_relative_error(df_gompertz_fit, out_complete_noblanks, 'mumax', 'mu_max')
print('the relative fitting error of mu_max is {:.4f} for logistic fits and {:.4f} for Gompertz fits'.format(A_log, A_gomp))
df_analysis.loc[2] = ['mu_max', mu_log, mu_gomp]

# compute errors in l
l_log = compute_median_relative_error(df_logistic_fit, out_complete_noblanks, 't0', 'l')
l_gomp = compute_median_relative_error(df_gompertz_fit, out_complete_noblanks, 't0', 'l')
print('the relative fitting error of l is {:.4f} for logistic fits and {:.4f} for Gompertz fits'.format(l_log, l_gomp))
df_analysis.loc[3] = ['l', l_log, l_gomp]


df_analysis = df_analysis.round(4)
df_analysis.to_excel('./synthetic_logistic_growth_curves_analysis.xlsx', index=False)
df_analysis

the relative fitting error of N0 is 0.0022 for logistic fits and 0.0000 for Gompertz fits
the relative fitting error of A is 0.0005 for logistic fits and 0.0000 for Gompertz fits
the relative fitting error of mu_max is 0.0005 for logistic fits and 0.0000 for Gompertz fits
the relative fitting error of l is 0.0013 for logistic fits and 0.0000 for Gompertz fits


,parameter,average_relative_error_logistic_model,average_relative_error_gompertz_model
0,N0,0.0022,0.0
1,A,0.0005,0.0
2,mu_max,0.0171,0.0
3,l,0.0013,0.0


# gompertz model, with noise

In [12]:
out_complete_clean = pd.read_excel('./synthetic_gompertz_growth_curves.xlsx').iloc[:, 4:]


In [13]:
measurement_noise = 0.05
out_complete_clean_noised = out_complete_clean.copy()
shape_noise = [out_complete_clean_noised.shape[0], out_complete_clean_noised.shape[1] - 1]
out_complete_clean_noised.iloc[:, 1:] = out_complete_clean_noised.iloc[:, 1:] + np.random.normal(0, measurement_noise, shape_noise)
out_complete_clean_noised.to_excel('./synthetic_gompertz_growth_curves_dgc_ready_noised.xlsx', index=False)

### compare to Dashing Growth Curve fits
Fit synthetic growth data with Dashing Growth Curves (conventional Gompertz function) and determine the error between estimated and true parameters.

As a sanity check, fit synthetic data also with the conventional Logistic model. The Logistic fits should be worse than the Logistic fits.

In [ ]:
# load data exported from Dashing Growth Curves:
df_logistic_fit = pd.read_csv('./synthetic_gompertz_data_noised_logistic_fits.csv', sep='\t').iloc[n_blanks:, :]
df_gompertz_fit = pd.read_csv('./synthetic_gompertz_data_noised_gompertz_fits.csv', sep='\t').iloc[n_blanks:, :]
out_complete_noblanks = pd.read_excel('./synthetic_gompertz_growth_curves.xlsx').iloc[n_blanks:, :]


# error functions
def compute_rmse(df_1, df_2, column_name_1, column_name_2):
    return np.sqrt((df_1[column_name_1] - df_2[column_name_2]).pow(2).sum()/df_1.shape[0])

def compute_median_relative_error(df_1, df_2, column_name_1, column_name_2):
    # return np.abs((df_1[column_name_1] - df_2[column_name_2]) / df_2[column_name_2]).sum()/df_1.shape[0]
    return np.median(np.abs((df_1[column_name_1] - df_2[column_name_2]) / df_2[column_name_2]))


df_analysis = pd.DataFrame(columns = ['parameter', 'average_relative_error_logistic_model', 'average_relative_error_gompertz_model'])

# compute errors in N0
N0_log = compute_median_relative_error(df_logistic_fit, out_complete_noblanks, 'N0', 'N0')
N0_gomp = compute_median_relative_error(df_gompertz_fit, out_complete_noblanks, 'N0', 'N0')
print('the relative fitting error of N0 is {:.4f} for logistic fits and {:.4f} for Gompertz fits'.format(N0_log, N0_gomp))
df_analysis.loc[0] = ['N0', N0_log, N0_gomp]

# compute errors in A
A_log = compute_median_relative_error(df_logistic_fit, out_complete_noblanks, 'A', 'A')
A_gomp = compute_median_relative_error(df_gompertz_fit, out_complete_noblanks, 'A', 'A')
print('the relative fitting error of A is {:.4f} for logistic fits and {:.4f} for Gompertz fits'.format(A_log, A_gomp))
df_analysis.loc[1] = ['A', A_log, A_gomp]

# compute errors in mu_max
mu_log = compute_median_relative_error(df_logistic_fit, out_complete_noblanks, 'mumax', 'mu_max')
mu_gomp = compute_median_relative_error(df_gompertz_fit, out_complete_noblanks, 'mumax', 'mu_max')
print('the relative fitting error of mu_max is {:.4f} for logistic fits and {:.4f} for Gompertz fits'.format(A_log, A_gomp))
df_analysis.loc[2] = ['mu_max', mu_log, mu_gomp]

# compute errors in l
l_log = compute_median_relative_error(df_logistic_fit, out_complete_noblanks, 't0', 'l')
l_gomp = compute_median_relative_error(df_gompertz_fit, out_complete_noblanks, 't0', 'l')
print('the relative fitting error of l is {:.4f} for logistic fits and {:.4f} for Gompertz fits'.format(l_log, l_gomp))
df_analysis.loc[3] = ['l', l_log, l_gomp]


df_analysis = df_analysis.round(4)
df_analysis.to_excel('./synthetic_logistic_growth_curves_noised_analysis.xlsx', index=False)
df_analysis